# Pose Extractor

This notebook extracts estimated pose using a pre-trained OpenPose[1,2] model. 

 1. Extract frames for a given video (fps)
 2. Collect pose keypoints for all frames

#### References:

[1] *OpenPose: Realtime Multi-Person 2D Pose Estimation using Part Affinity Fields*, Cao et al.

[2] https://github.com/CMU-Perceptual-Computing-Lab/openpose

In [ ]:
videos_dir = '/usr/local/data02/zahra/datasets/Tempuckey/videos'
frames_dir = '/usr/local/data02/zahra/datasets/Tempuckey/frames'
op_output_dir = '/usr/local/data02/zahra/datasets/Tempuckey/feats/openpose'

root_dir = '/usr/local/data01/zahra/repos/VideoFeatExtratotor'

## 1. Extract frames from a given video (fps)

In [ ]:
from utils.sys_utils import *
from utils.video_utils import *

In [ ]:
completed_vids = get_subfolders_path(frames_dir)
completed_vids = set([v.split('/')[-1] for v in completed_vids])

In [ ]:
videos = get_files_path(videos_dir)

# extract frames
for v in videos:
    vname = v.replace('/','.').split('.')[-2]
    
    if vname in completed_vids:
        print('{} was previously completed\n'.format(vname))
        continue
        
    f = '{}/{}'.format(frames_dir, vname)
    create_folder(f)
    
    vc = cv2.VideoCapture(v)
    count = get_video_frames(vc, f)
    print('stored {} frames at {}\n'.format(count, f))


## 2. Collect keypoints for all the frames in a clip 
### repeats for all clips and store heatmaps, keypoints, and ouput imgs for all frames in each clip

- ### <span style="color:green"> **2.1**</span> obtain joint heatmaps 
  - returns $H_j^t[x,y]$
  - in our BODY\_25 model, each image 1,2,...,25 contains all the same joints detected for everyone in the image

In [ ]:
# reference: openpose python api
import numpy as np
import pandas as pd
from openpose import pyopenpose as op
# setup the open pose model
params = dict()
params["model_folder"] = root_dir + "/models/"
params["heatmaps_add_parts"] = True
params["heatmaps_add_bkg"] = False
params["heatmaps_add_PAFs"] = False

# Starting OpenPose
opWrapper = op.WrapperPython()
opWrapper.configure(params)
opWrapper.start()

datum = op.Datum()

In [ ]:
def store_op_output(poseHeatMaps_t, outputImages_t, keypoints_t, beg_frame_idx, end_frame_idx, vname, op_output_dir):
    
    beg_frame_idx = str(beg_frame_idx).zfill(5)
    end_frame_idx = str(end_frame_idx).zfill(5)
    
    # num_frames, num_keypoints/num_joints , img_width, img_height => dims = (T, 25, w, h)
    # i.e. for image frame 0 we have 25 heatmaps (one per keypoint or joint)
    output_path = '{}/{}_heatmaps_T_25_w_h_{}_{}'.format(op_output_dir, vname, beg_frame_idx, end_frame_idx)

    print('{}'.format(output_path))
    np.save(output_path , np.array(poseHeatMaps_t))

    # outputImages_t[0].shape => (720, 1280, 3) 
    # one RGB image with dims = (W,H,3) per frame. and a total of T frames
    output_path = '{}/{}_posed_imgs_T_w_h_3_{}_{}'.format(op_output_dir, vname, beg_frame_idx, end_frame_idx)

    print('{}'.format(output_path))
    np.save(output_path , np.array(outputImages_t))

    # keypoints_t dims = (T, num_persons, num_keypoints, 3) => where 3 is [x,y,confidence] for each detected keypoint on a person
    output_path = '{}/{}_keypoints_T_persons_25_3_{}_{}'.format(op_output_dir, vname, beg_frame_idx, end_frame_idx)

    print('{}'.format(output_path))
    np.save(output_path , np.array(keypoints_t))
    print()


In [ ]:
## get path to all the videos
videos = get_files_path(videos_dir)

## get list of test set video for which we need to compute openpose estimations
labels_dir = '/usr/local/data02/zahra/datasets/Tempuckey/labels/tempuckey_video_info_gt_labels_split.csv'
labels_df = pd.read_csv(labels_dir)

test_set_video_names = list(labels_df[labels_df['split'] == 'test'].video_name)
test_set_video_names = set([v.split('.')[0] for v in test_set_video_names])

## get list of already completed (i.e. with extracted openpose)
completed_video_names = get_files_path(op_output_dir)
completed_video_names = set(['_'.join(v.split('/')[-1].split('_')[:3]) for v in completed_video_names])

## pick the subset of videos that belong to test set and were not previously completed
videos_to_process = []
for v in videos:
    vname = v.split('/')[-1].split('.')[0]
    if vname in test_set_video_names and vname not in completed_video_names:
        videos_to_process.append(v)
        
# videos = videos_to_process

In [ ]:
if videos_to_process == []:
    print('openpose for all test videos was extracted!')

In [ ]:
# videos = get_files_path(videos_dir)

win = 320 # pack 320 frames into one npy file 

# extract pose heatmap and keypoints for each frame
for v in videos:
    vname = v.replace('/','.').split('.')[-2]
    
    if vname in completed_vids:
        print('{} was previously completed'.format(vname))
        continue
        
    f = '{}/{}'.format(frames_dir, vname)
    frames = get_files_path(f)
    
    poseHeatMaps_t = []
    outputImages_t = []
    keypoints_t = []

    # framewise pose, heatmap, keypoint extraction
    for i in range(len(frames)):
        img = cv2.imread(frames[i])

        datum.cvInputData = img
        opWrapper.emplaceAndPop([datum])

        poseHeatMaps_t.append(datum.poseHeatMaps.copy())
        outputImages_t.append(datum.cvOutputData.copy())
        keypoints_t.append(datum.poseKeypoints.copy())
        
        if (i+1)%win == 0:
            
            beg_frame_idx = i - (win-1)
            end_frame_idx = i

            store_op_output(poseHeatMaps_t, outputImages_t, keypoints_t, beg_frame_idx, end_frame_idx, vname, op_output_dir)
            
            poseHeatMaps_t = []
            outputImages_t = []
            keypoints_t = []
            
    if i > end_frame_idx:
        
        beg_frame_idx = end_frame_idx
        end_frame_idx = i

        store_op_output(poseHeatMaps_t, outputImages_t, keypoints_t, beg_frame_idx, end_frame_idx, vname, op_output_dir)


In [ ]:
print('datum.poseHeatMaps.shape: \t', datum.poseHeatMaps.shape) #25_body_joints

# heat maps storing format: body parts + background + PAFs
print('datum.poseKeypoints.shape: \t', datum.poseKeypoints.shape) # (num_persons, num_joints_25, (x,y,confidence) )

print('datum.cvInputData.shape: \t', datum.cvInputData.shape)